# Examples for Behavior Cloning Dataset Generation and Usage in Training

This notebook contains convenient examples demonstrating use of dataset generation and training pipeline functions.

1. Start off by running the first cell below.
2. Then, proceed in order. Some cells may be able to be skipped depending on their content.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.absolute()))

dataset_dir_path = Path('D:/bc-train-data-test')

### Generating a Bimanual Behavior Cloning Dataset

Here, we generate a dataset of (BimanualObs, BimanualAction) samples for the "pass block" task using policy.PrivilegedPolicy.
This takes a while, so I've implemented convenient mechanisms for resuming data generation between sessions.
The resulting output files can be accessed again with the BimanualDataset and HumanReadableBimanualDataset classes, as shown later.

In [2]:
from train.dataset import generate_bimanual_dataset

generate_bimanual_dataset(
  save_dir=dataset_dir_path,
  total_sample_count=10000,
  max_steps_per_rollout=600,
  skip_frames=2,
  camera_dims=(128, 128),
  resume=True
)

Bimanual dataset save directory is set to `D:\bc-train-data-test`.
Resuming from sample 3749/10000.


Attempting rollout 27.: 100%|██████████| 600/600 [00:45<00:00, 13.20it/s]


 - Rollout failed. Discarded 200 samples at 2025-10-24 16:09:25.052801. (3749/10000)


Attempting rollout 27.: 100%|██████████| 600/600 [00:51<00:00, 11.69it/s]


 - Rollout failed. Discarded 200 samples at 2025-10-24 16:10:17.193238. (3749/10000)


...

 - Rollout failed. Discarded 200 samples at 2025-10-24 16:14:25.156023. (4005/10000)


Attempting rollout 29.: 100%|██████████| 600/600 [01:46<00:00,  5.63it/s]


 - Rollout failed. Discarded 200 samples at 2025-10-24 16:16:12.710979. (4005/10000)


Attempting rollout 29.:  96%|█████████▌| 574/600 [01:11<00:03,  8.08it/s]


KeyboardInterrupt: 

### Viewing a Rollout from the Generated Dataset

In [5]:
from robot.visualize import save_frames_to_video
from train.dataset import HumanReadableBimanualDataset

dataset = HumanReadableBimanualDataset(dataset_dir_path)
rollout_number = 7
rollout_length = dataset.metadata.rollout_lengths[rollout_number]
rollout_start = sum(dataset.metadata.rollout_lengths[:rollout_number])

os.makedirs('out', exist_ok=True)
left_wrist_video_path = f'out/left_wrist_rollout_{rollout_number}.mp4'
right_wrist_video_path = f'out/right_wrist_rollout_{rollout_number}.mp4'
save_frames_to_video([dataset[i][0].visual[0] for i in range(rollout_start, rollout_start + rollout_length)], left_wrist_video_path)
save_frames_to_video([dataset[i][0].visual[1] for i in range(rollout_start, rollout_start + rollout_length)], right_wrist_video_path)

In [6]:
from IPython.display import Video
Video(left_wrist_video_path, width=400, height=400)

In [7]:
from IPython.display import Video
Video(right_wrist_video_path, width=400, height=400)

In [22]:
import os
import torch
import torch.nn as nn
from torch import Tensor
from robot.sim import JOINT_OBSERVATION_SIZE, BimanualObs
from train.dataset import HumanReadableBimanualDataset
from train.train_utils import Logs
from train.trainer import BCTrainer, BimanualActor

# example bimanual actor class
class ExampleModel(BimanualActor):
  def __init__(self, observation_size: int, action_size: int, hidden_size: int = 256):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(observation_size, hidden_size),
      nn.ReLU(),
      nn.Linear(hidden_size, hidden_size),
      nn.ReLU(),
      nn.Linear(hidden_size, action_size)
    )

  def forward(self, obs: BimanualObs) -> Tensor:
    x = torch.cat((torch.from_numpy(obs.visual.flatten()), torch.from_numpy(obs.qpos.array)))
    return self.layers(x)

# load dataset and set up trainer
dataset = HumanReadableBimanualDataset(dataset_dir_path)
os.makedirs('out/training-output', exist_ok=True)
logs = Logs('out/training-output')
new_job = logs.create_new_job(tag='example')

# instantiate model
input_size = dataset.metadata.observation_size - JOINT_OBSERVATION_SIZE  # exclude qvel observation
output_size = dataset.metadata.action_size
model = ExampleModel(input_size, output_size)

# train with behavior cloning objective
dataset.metadata.sample_count = 200  # hacky way to shorten dataset for this example
trainer = BCTrainer(dataloader=dataset, checkpoint_frequency=1, job=new_job)
trainer.train(model, 10)

Training model for 10 epochs.


Epoch 0: 100%|██████████| 200/200 [00:18<00:00, 11.01it/s]


 - Epoch 0 loss: 206.5061


Epoch 1: 100%|██████████| 200/200 [00:14<00:00, 13.97it/s]


 - Epoch 1 loss: 13.6402


...

 - Epoch 7 loss: 26.7064


Epoch 8: 100%|██████████| 200/200 [00:14<00:00, 14.09it/s]


 - Epoch 8 loss: 26.5800


Epoch 9: 100%|██████████| 200/200 [00:14<00:00, 13.41it/s]


 - Epoch 9 loss: 26.5050
